In [1]:
import pandas as pd
from glob import glob

from jointly_hic.notebook_utils.encode_utils import get_signal
from jointly_hic.hdf5db.hdf5db import (
    load_bins,
    load_experiment_metadata,
    add_bigwig_track,
    load_track_metadata,
    load_tracks,
    JointDb,
)

In [2]:
EMBEDDINGS = "/home/reimonnt/hetchrom-hardening-march/joint-embeddings/output_2025_03_17_15_14_PCA-32_50000bp_hg38_post_processed.pq"
stage_mapper = {
    "DE.mcool": "DE",
    "ESC.mcool": "ESC",
    "HB.mcool": "HB",
    "iHEP.mcool": "iHEP",
    "mHEP.mcool": "mHEP",
}

In [3]:
df = pd.read_parquet(EMBEDDINGS)
df["stage"] = df["filename"].map(stage_mapper)
df["biosample"] = df["filename"].map(stage_mapper)
df

,chrom,start,end,weight,good_bin,filename,PCA1,PCA2,PCA3,PCA4,...,leiden_0_8_n500,leiden_1_0_n500,umap1_n30,umap2_n30,umap1_n100,umap2_n100,umap1_n500,umap2_n500,stage,biosample
0,chr1,0,50000,NaN,False,DE.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,DE
1,chr1,50000,100000,NaN,False,DE.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,DE
2,chr1,100000,150000,NaN,False,DE.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,DE
3,chr1,150000,200000,NaN,False,DE.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,DE
4,chr1,200000,250000,NaN,False,DE.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287540,chr22,50600000,50650000,0.005226,True,mHEP.mcool,0.009316,0.008908,0.002670,-0.000418,...,4.0,10.0,14.590994,6.275971,11.685134,7.152789,11.534708,7.622754,mHEP,mHEP
287541,chr22,50650000,50700000,0.005636,True,mHEP.mcool,0.009301,0.009699,0.002085,-0.000512,...,4.0,10.0,14.532645,6.173001,11.567213,7.051041,11.564776,7.499127,mHEP,mHEP
287542,chr22,50700000,50750000,0.005296,True,mHEP.mcool,0.010164,0.012430,0.001861,-0.000552,...,4.0,10.0,14.315054,5.833733,11.345654,6.827947,11.308563,7.347744,mHEP,mHEP
287543,chr22,50750000,50800000,NaN,False,mHEP.mcool,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mHEP,mHEP


In [4]:
df.to_parquet("../joint-embeddings/hepdiff_embeddings_prepared.pq")

In [5]:
! python /home/reimonnt/jointly-hic/jointly_hic/hdf5db/experiments2yaml.py --parquet-file ../joint-embeddings/hepdiff_embeddings_prepared.pq --accession-column stage --metadata-columns filename biosample --yaml-file hepdiff-jointdb-config.yaml

In [6]:
! python /home/reimonnt/jointly-hic/jointly_hic/hdf5db/hdf5db.py --experiments hepdiff-jointdb-config.yaml --embeddings ../joint-embeddings/hepdiff_embeddings_prepared.pq --accession stage --output hepdiff-jointdb.hdf5

In [7]:
# Load bin data
jointdb = "hepdiff-jointdb.hdf5"
bins = load_bins(jointdb)
bins

,chrom,start,end,good_bin,bin_name
0,chr1,0,50000,False,chr1:0-50000
1,chr1,50000,100000,False,chr1:50000-100000
2,chr1,100000,150000,False,chr1:100000-150000
3,chr1,150000,200000,False,chr1:150000-200000
4,chr1,200000,250000,False,chr1:200000-250000
...,...,...,...,...,...
57504,chr9,138150000,138200000,False,chr9:138150000-138200000
57505,chr9,138200000,138250000,False,chr9:138200000-138250000
57506,chr9,138250000,138300000,False,chr9:138250000-138300000
57507,chr9,138300000,138350000,False,chr9:138300000-138350000


In [8]:
# Load tracks
track_files = glob("/net/projects/4dn2_hepdiff/cutnrun/*/results/*/signal/*/*.bigwig")
trackdf = pd.DataFrame(track_files, columns=["filename"])

stage_mapper = {
    "02_DE": "DE",
    "01_H1": "ESC",
    "03_HB": "HB",
    "04_iHEP": "iHEP",
    "05_mHEP": "mHEP",
}


def get_stage(fn):
    vals = fn.strip().split("/")
    exp = vals[7]
    details = exp.split(".")
    return stage_mapper[details[0]]


def get_target(fn):
    vals = fn.strip().split("/")
    exp = vals[7]
    details = exp.split(".")
    return details[1]


def get_type(fn):
    vals = fn.strip().split(".")
    return vals[-3]


def get_exp(fn):
    vals = fn.strip().split("/")
    exp = vals[7] + "." + vals[9]
    return exp


def get_replicate(fn):
    vals = fn.strip().split("/")
    return vals[9]


trackdf["biosample"] = trackdf["filename"].map(get_stage)
trackdf["assay"] = trackdf["filename"].map(get_target)
trackdf["type"] = trackdf["filename"].map(get_type)
trackdf["experiment"] = trackdf["filename"].map(get_exp)
trackdf["replicate"] = trackdf["filename"].map(get_replicate)
trackdf["accession"] = (
    trackdf["biosample"]
    + "_"
    + trackdf["assay"]
    + "_"
    + trackdf["replicate"]
    + "_"
    + trackdf["type"]
)
trackdf

,filename,biosample,assay,type,experiment,replicate,accession
0,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K9me2,pval,01_H1.H3K9me2.B1_Mohan1.B3_Mohan2.rep1,rep1,ESC_H3K9me2_rep1_pval
1,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K9me2,fc,01_H1.H3K9me2.B1_Mohan1.B3_Mohan2.rep1,rep1,ESC_H3K9me2_rep1_fc
2,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K9me2,pval,01_H1.H3K9me2.B1_Mohan1.B3_Mohan2.pooled-rep,pooled-rep,ESC_H3K9me2_pooled-rep_pval
3,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K9me2,fc,01_H1.H3K9me2.B1_Mohan1.B3_Mohan2.pooled-rep,pooled-rep,ESC_H3K9me2_pooled-rep_fc
4,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K9me2,pval,01_H1.H3K9me2.B1_Mohan1.B3_Mohan2.rep2,rep2,ESC_H3K9me2_rep2_pval
...,...,...,...,...,...,...,...
133,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K27me3,pval,01_H1.H3K27me3.B1_Mohan1.B3_Mohan2.pooled-rep,pooled-rep,ESC_H3K27me3_pooled-rep_pval
134,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K27me3,fc,01_H1.H3K27me3.B1_Mohan1.B3_Mohan2.rep1,rep1,ESC_H3K27me3_rep1_fc
135,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K27me3,pval,01_H1.H3K27me3.B1_Mohan1.B3_Mohan2.rep1,rep1,ESC_H3K27me3_rep1_pval
136,/net/projects/4dn2_hepdiff/cutnrun/2024-06-13_...,ESC,H3K27me3,pval,01_H1.H3K27me3.B1_Mohan1.B3_Mohan2.rep2,rep2,ESC_H3K27me3_rep2_pval


In [9]:
trackdf[["biosample", "assay", "type", "accession", "experiment", "filename"]].to_csv(
    "track_metadata.txt", sep="\t", index=False
)

In [10]:
for i, row in trackdf.iterrows():
    # print(f"Adding {row["accession"]} to JointDb...")
    signal = get_signal(row["filename"], bins)
    add_bigwig_track(
        jointdb,
        signal,
        row["accession"],
        row["assay"],
        row["experiment"],
        row["biosample"],
    )

Adding ESC_H3K9me2_rep1_pval to JointDb...
Adding ESC_H3K9me2_rep1_fc to JointDb...
Adding ESC_H3K9me2_pooled-rep_pval to JointDb...
Adding ESC_H3K9me2_pooled-rep_fc to JointDb...
Adding ESC_H3K9me2_rep2_pval to JointDb...
Adding ESC_H3K9me2_rep2_fc to JointDb...
Adding DE_CTCF_rep1_fc to JointDb...
Adding DE_CTCF_rep1_pval to JointDb...
Adding mHEP_H3K4me3_rep1_fc to JointDb...
Adding mHEP_H3K4me3_rep1_pval to JointDb...
Adding iHEP_RAD21_rep1_pval to JointDb...
Adding iHEP_RAD21_rep1_fc to JointDb...
Adding HB_H3K4me1_rep1_fc to JointDb...
Adding HB_H3K4me1_rep1_pval to JointDb...
Adding DE_H3K9me3_rep1_pval to JointDb...
Adding DE_H3K9me3_rep1_fc to JointDb...
Adding mHEP_H3K4me1_rep1_fc to JointDb...
Adding mHEP_H3K4me1_rep1_pval to JointDb...
Adding HB_H3K4me3_rep1_fc to JointDb...
Adding HB_H3K4me3_rep1_pval to JointDb...
Adding DE_H3K9me2_rep1_pval to JointDb...
Adding DE_H3K9me2_rep1_fc to JointDb...
Adding HB_H4K20me3_rep1_fc to JointDb...
Adding HB_H4K20me3_rep1_pval to Joint

In [11]:
# List of file paths for each stage and replicate
file_paths = [
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP1.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP1.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP2.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP2.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP1.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP1.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP2.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP2.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/HB_REP1.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/HB_REP1.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/HB_REP2.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/HB_REP2.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/iHEP_REP1.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/iHEP_REP1.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/iHEP_REP2.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/iHEP_REP2.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/mHEP_REP1.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/mHEP_REP1.reverse.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/mHEP_REP2.forward.bigWig",
    "/net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/mHEP_REP2.reverse.bigWig",
]

# Dictionary to store signals for each stage
stage_signals = {}

# Loop over each file to compute signal
for file_path in file_paths:
    print(f"Loading {file_path}")
    # Extract information from the filename
    filename = file_path.split("/")[-1]
    parts = filename.split("_")
    stage = parts[0]
    replicate = parts[1].split(".")[0]
    direction = parts[1].split(".")[1]

    # Call get_signal function for each file
    signal = get_signal(file_path, bins)

    # Initialize stage entry in the dictionary if not present
    if stage not in stage_signals:
        stage_signals[stage] = {"forward": None, "reverse": None}

    # Sum signals for REP1 and REP2
    if stage_signals[stage][direction] is None:
        stage_signals[stage][direction] = signal
    else:
        stage_signals[stage][direction] += signal

# Combine forward and reverse signals for each stage and call add_bigwig_track
for stage, directions in stage_signals.items():
    # Summing forward and reverse signals
    combined_signal = directions["forward"] + directions["reverse"]

    # Define accession based on the stage name (for tracking purposes)
    accession = f"{stage}_RNA_fc"
    print(f"Saving {accession}")

    # Call add_bigwig_track function with the combined signal
    add_bigwig_track(jointdb, combined_signal, accession, "RNA", accession, stage)

Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP1.forward.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP1.reverse.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP2.forward.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/DE_REP2.reverse.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP1.forward.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP1.reverse.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP2.forward.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/ESC_REP2.reverse.bigWig
Loading /net/projects/4dn2_hepdiff/rnaseq/2022-11-24_nf-core/results/star_salmon/bigwig/HB_REP1.forward.bigW

In [12]:
# Define the ATAC-seq file paths
atacseq = {
    "ESC-1": "/net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/01_ESC/signal/rep1/4DNFIVWIBHN4.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "ESC-2": "/net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/01_ESC/signal/rep2/4DNFIRULVCWY.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "DE-1": "/net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/02_DE/signal/rep1/4DNFI85N5FRY.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "DE-2": "/net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/02_DE/signal/rep2/4DNFIWNSGYUF.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "HB-1": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/03_HB/signal/rep1/HB_ATAC_S1_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "HB-2": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/03_HB/signal/rep2/ATAC-HB_S1_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "iHEP-1": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/04_iHEP/signal/rep1/iHEP_ATAC_S2_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "iHEP-2": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/04_iHEP/signal/rep2/ATAC-iHEP_S2_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "mHEP-1": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/05_mHEP/signal/rep1/ATAC-mHEP_S3_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
    "mHEP-2": "/net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/05_mHEP/signal/rep2/ATAC-mHEP-rep1_S4_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig",
}

# Dictionary to store combined signals for each stage
stage_signals = {}

# Loop over each entry in the atacseq dictionary
for key, filepath in atacseq.items():
    print(f"Loading {filepath}")
    # Split the key to extract the stage and replicate
    stage, replicate = key.split("-")

    # Compute the signal for the file
    signal = get_signal(filepath, bins)

    # Initialize the stage entry if it doesn't exist
    if stage not in stage_signals:
        stage_signals[stage] = None

    # Sum the signals from both replicates for each stage
    if stage_signals[stage] is None:
        stage_signals[stage] = signal
    else:
        stage_signals[stage] += signal

# Call add_bigwig_track for each stage with the combined signal
for stage, combined_signal in stage_signals.items():
    accession = f"{stage}_ATAC_fc"
    print(f"Loading {accession}")
    add_bigwig_track(jointdb, combined_signal, accession, "ATAC", accession, stage)

Loading /net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/01_ESC/signal/rep1/4DNFIVWIBHN4.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/01_ESC/signal/rep2/4DNFIRULVCWY.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/02_DE/signal/rep1/4DNFI85N5FRY.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-04-22_encode-pipeline/results/02_DE/signal/rep2/4DNFIWNSGYUF.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/03_HB/signal/rep1/HB_ATAC_S1_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-08-08_encode-pipeline/results/03_HB/signal/rep2/ATAC-HB_S1_R1_001.trim.srt.nodup.no_chrM_MT.tn5.fc.signal.bigwig
Loading /net/projects/4dn2_hepdiff/atacseq/2024-08-0

In [14]:
# SON TSA-Seq
metadata = pd.read_table(
    "~/TSA-seq/metadata_2024-09-03-14h-40m.tsv", sep="\t", skiprows=1
)
metadata["path"] = [
    f"/home/reimonnt/TSA-seq/{acc}.bw" for acc in metadata["File Accession"]
]
metadata["biosample"] = [x.split(" ")[0] for x in metadata["Biosource"]]
metadata["biosample"] = metadata["biosample"].map(
    {"K562": "K562", "HFFc6": "HFF", "H1-hESC": "ESC", "HCT116": "HCT116"}
)
metadata

for ix, row in metadata.iterrows():
    accession = row["File Accession"]
    biosample = row["biosample"]
    path = row["path"]
    print(f"{biosample} -- {accession} -- {path}")
    signal = get_signal(
        path,
        bins,
    )
    add_bigwig_track(
        jointdb,
        signal,
        accession,
        "SON-TSA-Seq",
        accession,
        biosample,
    )

K562 -- 4DNFIWRT1RIZ -- /home/reimonnt/TSA-seq/4DNFIWRT1RIZ.bw
K562 -- 4DNFIDR98YO6 -- /home/reimonnt/TSA-seq/4DNFIDR98YO6.bw
K562 -- 4DNFIN6J6X2A -- /home/reimonnt/TSA-seq/4DNFIN6J6X2A.bw
HFF -- 4DNFI6FTPH5V -- /home/reimonnt/TSA-seq/4DNFI6FTPH5V.bw
HFF -- 4DNFIO4EE1OU -- /home/reimonnt/TSA-seq/4DNFIO4EE1OU.bw
K562 -- 4DNFIVH32CG2 -- /home/reimonnt/TSA-seq/4DNFIVH32CG2.bw
K562 -- 4DNFI2WK5IVI -- /home/reimonnt/TSA-seq/4DNFI2WK5IVI.bw
ESC -- 4DNFI625PP2A -- /home/reimonnt/TSA-seq/4DNFI625PP2A.bw
ESC -- 4DNFIFKMOD1L -- /home/reimonnt/TSA-seq/4DNFIFKMOD1L.bw
K562 -- 4DNFI2HW7CWH -- /home/reimonnt/TSA-seq/4DNFI2HW7CWH.bw
HCT116 -- 4DNFI9GKAT7E -- /home/reimonnt/TSA-seq/4DNFI9GKAT7E.bw
HCT116 -- 4DNFIBY8G6RZ -- /home/reimonnt/TSA-seq/4DNFIBY8G6RZ.bw
HCT116 -- 4DNFI8HIU45G -- /home/reimonnt/TSA-seq/4DNFI8HIU45G.bw
HCT116 -- 4DNFIHXJ1RCP -- /home/reimonnt/TSA-seq/4DNFIHXJ1RCP.bw


In [16]:
# LC-Hi-C tracks
signal = get_signal(
    "/home/reimonnt/heterocrhom-hardening/LC-Hi-C/DL-20240611-LCHCSeq-HB-2hPD-DpnII-R3-T1__hg38.hg38.exclude1kb.10000_range2Mb_LOS.bw",
    bins,
)
add_bigwig_track(
    jointdb,
    signal,
    "DL-20240611-LCHCSeq-HB-2hPD-DpnII",
    "LC-Hi-C",
    "DL-20240611-LCHCSeq-HB-2hPD-DpnII",
    "HB",
)

In [17]:
signal = get_signal(
    "/home/reimonnt/heterocrhom-hardening/LC-Hi-C/H1ESC-nPI-DpnII__hg38.hg38.exclude1kb.10000_range2Mb_LOS.bw",
    bins,
)
add_bigwig_track(
    jointdb,
    signal,
    "H1ESC-nPI-DpnII",
    "LC-Hi-C",
    "H1ESC-nPI-DpnII",
    "ESC",
)